# Create multi trending from canvases with pads
> Goal is to trend and check canvases with multiple pads, such as the ones we get from the cluster task


<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-libraries" data-toc-modified-id="Import-libraries-1">Import libraries</a></span></li><li><span><a href="#Read-ROOT-files" data-toc-modified-id="Read-ROOT-files-2">Read ROOT files</a></span><ul class="toc-item"><li><span><a href="#Showing-the-canvases-we-want-to-trend" data-toc-modified-id="Showing-the-canvases-we-want-to-trend-2.1">Showing the canvases we want to trend</a></span></li></ul></li><li><span><a href="#Trend-and-check" data-toc-modified-id="Trend-and-check-3">Trend and check</a></span><ul class="toc-item"><li><span><a href="#Create-the-quality-matrix-to-keep-track-of-the-check-results" data-toc-modified-id="Create-the-quality-matrix-to-keep-track-of-the-check-results-3.1">Create the quality matrix to keep track of the check results</a></span></li><li><span><a href="#multiTrending" data-toc-modified-id="multiTrending-3.2">multiTrending</a></span></li><li><span><a href="#Draw-the-QualityMatrix" data-toc-modified-id="Draw-the-QualityMatrix-3.3">Draw the QualityMatrix</a></span></li></ul></li></ul></div>

## Import libraries

In [ ]:
%%capture
import ROOT
import glob
import math
import numpy as np
import pandas as pd
from IPython.display import display, Markdown
from IPython.display import display, HTML
from TPCQCVis.src.drawHistograms import *
from TPCQCVis.src.drawTrending import *
from TPCQCVis.src.drawMultiTrending import *
from TPCQCVis.src.checkHistograms import *
from TPCQCVis.src.checkTrending import *

In [ ]:
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>table {float:left;}</style>"))
%jsroot on

## Read ROOT files

In [ ]:
# Read the Root Files
path = "/cave/alice/data/2022/LHC22o/apass3/"
fileList = glob.glob(path+"*_QC.root")
fileList.sort()
#fileList = fileList[13:]
runList = [fileList[i][-14:-8] for i in range(len(fileList))]
rootDataFile=[]
for file in fileList:
    rootDataFile.append(ROOT.TFile.Open(file,"READ"))
print(runList)

### Showing the canvases we want to trend
Make sure you have these canvases in your *_QC.root file
The plotter should be new version when running (https://github.com/bulukutlu/TPCQCVis/commit/179e67ef0d4d4032d6a7dfcf4aa1ba4058790980)

In [ ]:
objectName = "c_ROCs_N_Clusters_1D"
c = rootDataFile[0].ClusterQC.Get(objectName)
c.Draw()

In [ ]:
# The pads in the canvas
c.GetListOfPrimitives().ls()

## Trend and check
### Create the quality matrix to keep track of the check results

In [ ]:
qualityDF = pd.DataFrame({'runNumber':runList})
names = []
for i in range(72):
    if i < 18*2: roc = "IROC-"
    else: roc = "OROC-"
    if i%(18*2) < 18: side = "A"
    else : side = "C"
    names.append(roc+side+str(i%18).zfill(2))

### multiTrending
For the checker I simply call the checkTrending function on each trend in the output canvas

In [ ]:
objectName="c_ROCs_N_Clusters_1D;1"
trending = "mean"
error = ""
[trends,canvas] = drawMultiTrending(objectName,rootDataFile,names=runList,namesFromRunList=True,
                               trend=trending,error=error,pads=True,normalize=True,
                               log="none",yAxisRange=[0,500])
for iPad,pad in enumerate(canvas.GetListOfPrimitives()):
    trend = trends[iPad]
    [qualities,pad] = checkTrending(trend,canvas=pad,thresholds={"GOOD":3,"MEDIUM":6,"BAD":9})
    qualityDF[names[iPad]] = qualities
    canvas.Update()
canvas.Draw()

### Draw the QualityMatrix

In [ ]:
myPalette = np.array([920, 414, 801, 633],dtype=np.int32)
ROOT.gStyle.SetPalette(4,myPalette)
ROOT.gStyle.SetGridStyle(1)
qualityLabels = {"NULL":0,"GOOD":1,"MEDIUM":2,"BAD":3}

canvas = ROOT.TCanvas("QualityMatrix","",1100,400)
canvas.SetLeftMargin(0.15)
canvas.SetBottomMargin(0.15)
canvas.SetGrid()

qualityHist = ROOT.TH2I("QualityMatrix","Quality Matrix Clusters",
                        len(qualityDF.index),min(qualityDF.index),max(qualityDF.index)+1,
                        len(qualityDF.columns)-1,0,len(qualityDF.columns)-1)
qualityHist.SetCanExtend(ROOT.TH1.kAllAxes)
qualityHist.SetStats(0)
for runIndex,run in enumerate(qualityDF.runNumber):
    qualityHist.GetXaxis().SetBinLabel(runIndex+1,str(run))
    qualityHist.GetXaxis().SetTickLength( 0.03)
    for checkIndex,check in enumerate(qualityDF.loc[:, qualityDF.columns != "runNumber"].columns):
        #print(checkIndex,runIndex)
        qualityHist.Fill(run,check,qualityLabels.get(qualityDF.iloc[runIndex][check]))

qualityHist.LabelsOption("u")
qualityHist.Draw("COL")
qualityHist.GetZaxis().SetRangeUser(-0.5,3.5);
qualityHist.LabelsOption("v")

canvas.Draw()